# Comparação de Modelos Vetoriais com PCA

Este notebook demonstra a comparação entre dois modelos vetoriais diferentes:
1. **TF-IDF**: Modelo clássico baseado em frequência de termos
2. **Sentence-BERT**: Modelo moderno baseado em transformers

Usaremos PCA (Principal Component Analysis) para reduzir a dimensionalidade e visualizar os vetores em 2D.

## 1. Instalação de Dependências

In [ ]:
# Instalar bibliotecas necessárias
!pip install -q sentence-transformers scikit-learn matplotlib numpy pandas

## 2. Importação de Bibliotecas

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sentence_transformers import SentenceTransformer
import warnings
warnings.filterwarnings('ignore')

# Configuração para melhor visualização
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

## 3. Definição do Conjunto de Textos

Vamos criar um conjunto de 10 sentenças em português, agrupadas por temas:
- Tecnologia e programação (3 sentenças)
- Esportes e futebol (3 sentenças)
- Culinária e comida (2 sentenças)
- Clima e natureza (2 sentenças)

In [ ]:
# Conjunto de textos com temas similares
textos = [
    # Tecnologia (IDs 0-2)
    "Python é uma linguagem de programação muito versátil e poderosa",
    "Machine learning permite que computadores aprendam com dados",
    "Inteligência artificial está transformando a tecnologia moderna",
    
    # Esportes (IDs 3-5)
    "O futebol é o esporte mais popular do Brasil",
    "Os jogadores treinam todos os dias para melhorar suas habilidades",
    "A Copa do Mundo é o maior evento esportivo do planeta",
    
    # Culinária (IDs 6-7)
    "A feijoada é um prato tradicional da culinária brasileira",
    "Chefs profissionais criam receitas deliciosas e inovadoras",
    
    # Clima (IDs 8-9)
    "O clima tropical caracteriza-se por temperaturas elevadas",
    "As chuvas intensas são comuns durante o verão"
]

# Labels para identificar os grupos
labels = [
    "Tecnologia", "Tecnologia", "Tecnologia",
    "Esportes", "Esportes", "Esportes",
    "Culinária", "Culinária",
    "Clima", "Clima"
]

print(f"Total de textos: {len(textos)}")
print("\nTextos e suas categorias:")
for i, (texto, label) in enumerate(zip(textos, labels)):
    print(f"[{i}] {label}: {texto}")

## 4. Modelo 1: TF-IDF (Term Frequency - Inverse Document Frequency)

TF-IDF é um modelo vetorial clássico que representa textos baseado na frequência dos termos, dando mais peso a palavras raras e menos peso a palavras comuns.

In [ ]:
# Criar e treinar o modelo TF-IDF
tfidf_vectorizer = TfidfVectorizer(
    max_features=100,  # Limitar features para visualização clara
    ngram_range=(1, 2),  # Unigramas e bigramas
    min_df=1
)

# Codificar os textos
tfidf_vectors = tfidf_vectorizer.fit_transform(textos).toarray()

print(f"Forma da matriz TF-IDF: {tfidf_vectors.shape}")
print(f"Dimensionalidade dos vetores: {tfidf_vectors.shape[1]}")
print(f"\nPrimeiras 5 features (palavras):")
feature_names = tfidf_vectorizer.get_feature_names_out()
print(feature_names[:5])

### Visualização dos Vetores TF-IDF (primeiras dimensões)

In [ ]:
# Mostrar alguns vetores TF-IDF
print("Exemplos de vetores TF-IDF (primeiras 10 dimensões):")
print("-" * 80)
for i in range(min(3, len(textos))):
    print(f"\nTexto {i}: '{textos[i][:50]}...'")
    print(f"Vetor: {tfidf_vectors[i][:10]}")

## 5. Modelo 2: Sentence-BERT

Sentence-BERT é um modelo moderno baseado em transformers que gera embeddings semânticos de alta qualidade, capturando o significado contextual das sentenças.

In [ ]:
# Carregar modelo Sentence-BERT (multilíngue - suporta português)
print("Carregando modelo Sentence-BERT...")
sbert_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
print("Modelo carregado com sucesso!")

# Codificar os textos
sbert_vectors = sbert_model.encode(textos, show_progress_bar=True)

print(f"\nForma da matriz Sentence-BERT: {sbert_vectors.shape}")
print(f"Dimensionalidade dos vetores: {sbert_vectors.shape[1]}")

### Visualização dos Vetores Sentence-BERT (primeiras dimensões)

In [ ]:
# Mostrar alguns vetores Sentence-BERT
print("Exemplos de vetores Sentence-BERT (primeiras 10 dimensões):")
print("-" * 80)
for i in range(min(3, len(textos))):
    print(f"\nTexto {i}: '{textos[i][:50]}...'")
    print(f"Vetor: {sbert_vectors[i][:10]}")

## 6. Aplicação de PCA para Redução de Dimensionalidade

Vamos reduzir os vetores de alta dimensionalidade para 2D usando PCA, permitindo visualização em um gráfico de dispersão.

In [ ]:
# PCA para TF-IDF
pca_tfidf = PCA(n_components=2, random_state=42)
tfidf_2d = pca_tfidf.fit_transform(tfidf_vectors)

print("PCA - TF-IDF:")
print(f"Variância explicada: {pca_tfidf.explained_variance_ratio_}")
print(f"Variância total explicada: {sum(pca_tfidf.explained_variance_ratio_):.2%}")

# PCA para Sentence-BERT
pca_sbert = PCA(n_components=2, random_state=42)
sbert_2d = pca_sbert.fit_transform(sbert_vectors)

print("\nPCA - Sentence-BERT:")
print(f"Variância explicada: {pca_sbert.explained_variance_ratio_}")
print(f"Variância total explicada: {sum(pca_sbert.explained_variance_ratio_):.2%}")

## 7. Visualização das Projeções PCA

Vamos visualizar como cada modelo agrupa os textos similares no espaço 2D.

In [ ]:
# Cores para cada categoria
cores_map = {
    "Tecnologia": "#FF6B6B",  # Vermelho
    "Esportes": "#4ECDC4",    # Azul turquesa
    "Culinária": "#FFD93D",   # Amarelo
    "Clima": "#95E1D3"        # Verde água
}

cores = [cores_map[label] for label in labels]

# Criar figura com dois subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

# Plot 1: TF-IDF
ax1.scatter(tfidf_2d[:, 0], tfidf_2d[:, 1], c=cores, s=200, alpha=0.6, edgecolors='black', linewidth=1.5)
for i, txt in enumerate(range(len(textos))):
    ax1.annotate(f'{i}', (tfidf_2d[i, 0], tfidf_2d[i, 1]), 
                ha='center', va='center', fontsize=10, fontweight='bold')
ax1.set_xlabel('Componente Principal 1', fontsize=12)
ax1.set_ylabel('Componente Principal 2', fontsize=12)
ax1.set_title('Modelo TF-IDF - Projeção PCA\n' + 
              f'Variância explicada: {sum(pca_tfidf.explained_variance_ratio_):.1%}', 
              fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Plot 2: Sentence-BERT
ax2.scatter(sbert_2d[:, 0], sbert_2d[:, 1], c=cores, s=200, alpha=0.6, edgecolors='black', linewidth=1.5)
for i, txt in enumerate(range(len(textos))):
    ax2.annotate(f'{i}', (sbert_2d[i, 0], sbert_2d[i, 1]), 
                ha='center', va='center', fontsize=10, fontweight='bold')
ax2.set_xlabel('Componente Principal 1', fontsize=12)
ax2.set_ylabel('Componente Principal 2', fontsize=12)
ax2.set_title('Modelo Sentence-BERT - Projeção PCA\n' + 
              f'Variância explicada: {sum(pca_sbert.explained_variance_ratio_):.1%}', 
              fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

# Legenda
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=cor, label=cat, edgecolor='black') 
                   for cat, cor in cores_map.items()]
fig.legend(handles=legend_elements, loc='upper center', ncol=4, 
           fontsize=11, bbox_to_anchor=(0.5, 0.98), frameon=True)

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.savefig('comparacao_modelos_vetoriais_pca.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Visualização salva como 'comparacao_modelos_vetoriais_pca.png'")

## 8. Análise de Similaridade

Vamos calcular a similaridade de cosseno entre os textos para cada modelo.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Calcular matrizes de similaridade
sim_tfidf = cosine_similarity(tfidf_vectors)
sim_sbert = cosine_similarity(sbert_vectors)

# Visualizar matrizes de similaridade
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Heatmap TF-IDF
im1 = ax1.imshow(sim_tfidf, cmap='YlOrRd', aspect='auto', vmin=0, vmax=1)
ax1.set_title('Matriz de Similaridade - TF-IDF', fontsize=14, fontweight='bold')
ax1.set_xlabel('Texto ID', fontsize=12)
ax1.set_ylabel('Texto ID', fontsize=12)
ax1.set_xticks(range(len(textos)))
ax1.set_yticks(range(len(textos)))
plt.colorbar(im1, ax=ax1, label='Similaridade de Cosseno')

# Adicionar valores na matriz
for i in range(len(textos)):
    for j in range(len(textos)):
        text = ax1.text(j, i, f'{sim_tfidf[i, j]:.2f}',
                       ha="center", va="center", color="black", fontsize=8)

# Heatmap Sentence-BERT
im2 = ax2.imshow(sim_sbert, cmap='YlOrRd', aspect='auto', vmin=0, vmax=1)
ax2.set_title('Matriz de Similaridade - Sentence-BERT', fontsize=14, fontweight='bold')
ax2.set_xlabel('Texto ID', fontsize=12)
ax2.set_ylabel('Texto ID', fontsize=12)
ax2.set_xticks(range(len(textos)))
ax2.set_yticks(range(len(textos)))
plt.colorbar(im2, ax=ax2, label='Similaridade de Cosseno')

# Adicionar valores na matriz
for i in range(len(textos)):
    for j in range(len(textos)):
        text = ax2.text(j, i, f'{sim_sbert[i, j]:.2f}',
                       ha="center", va="center", color="black", fontsize=8)

plt.tight_layout()
plt.savefig('matriz_similaridade_modelos.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Matrizes de similaridade salvas como 'matriz_similaridade_modelos.png'")

## 9. Comparação e Análise dos Resultados

In [ ]:
print("=" * 80)
print("ANÁLISE COMPARATIVA DOS MODELOS VETORIAIS")
print("=" * 80)

print("\n📊 CARACTERÍSTICAS DOS MODELOS:")
print("-" * 80)
print(f"\nTF-IDF:")
print(f"  • Dimensionalidade: {tfidf_vectors.shape[1]}")
print(f"  • Tipo: Baseado em frequência de termos")
print(f"  • Representação: Esparsa (muitos zeros)")
print(f"  • Variância PCA (2D): {sum(pca_tfidf.explained_variance_ratio_):.2%}")

print(f"\nSentence-BERT:")
print(f"  • Dimensionalidade: {sbert_vectors.shape[1]}")
print(f"  • Tipo: Baseado em transformers (deep learning)")
print(f"  • Representação: Densa (valores em todas dimensões)")
print(f"  • Variância PCA (2D): {sum(pca_sbert.explained_variance_ratio_):.2%}")

print("\n🎯 AGRUPAMENTO POR CATEGORIA:")
print("-" * 80)

# Analisar agrupamento intra-categoria
categorias_unicas = list(set(labels))
for cat in categorias_unicas:
    indices = [i for i, label in enumerate(labels) if label == cat]
    if len(indices) > 1:
        # Similaridade média intra-categoria
        sim_intra_tfidf = np.mean([sim_tfidf[i][j] for i in indices for j in indices if i != j])
        sim_intra_sbert = np.mean([sim_sbert[i][j] for i in indices for j in indices if i != j])
        
        print(f"\n{cat} (textos {indices}):")
        print(f"  • Similaridade média TF-IDF: {sim_intra_tfidf:.3f}")
        print(f"  • Similaridade média SBERT: {sim_intra_sbert:.3f}")

print("\n\n📝 OBSERVAÇÕES:")
print("-" * 80)
print("""
1. TF-IDF:
   - Foca em palavras-chave compartilhadas
   - Pode ter dificuldade com sinônimos
   - Mais sensível a vocabulário específico
   
2. Sentence-BERT:
   - Captura significado semântico profundo
   - Melhor com paráfrases e sinônimos
   - Agrupa melhor textos conceitualmente similares
   
3. Nos gráficos PCA:
   - Textos da mesma categoria devem aparecer próximos
   - SBERT geralmente produz agrupamentos mais coesos
   - TF-IDF pode separar mais baseado em vocabulário
""")
print("=" * 80)

## 10. Visualização Detalhada por Categoria

In [ ]:
# Criar visualização com linhas conectando textos da mesma categoria
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

# Função para desenhar linhas entre pontos da mesma categoria
def draw_category_connections(ax, coords, labels, cores_map):
    categorias = list(set(labels))
    for cat in categorias:
        indices = [i for i, label in enumerate(labels) if label == cat]
        if len(indices) > 1:
            # Desenhar linhas entre todos os pontos da categoria
            for i in range(len(indices)):
                for j in range(i+1, len(indices)):
                    idx1, idx2 = indices[i], indices[j]
                    ax.plot([coords[idx1, 0], coords[idx2, 0]], 
                           [coords[idx1, 1], coords[idx2, 1]], 
                           color=cores_map[cat], alpha=0.3, linewidth=2, linestyle='--')

# Plot 1: TF-IDF com conexões
draw_category_connections(ax1, tfidf_2d, labels, cores_map)
ax1.scatter(tfidf_2d[:, 0], tfidf_2d[:, 1], c=cores, s=200, alpha=0.6, edgecolors='black', linewidth=1.5)
for i, txt in enumerate(range(len(textos))):
    ax1.annotate(f'{i}', (tfidf_2d[i, 0], tfidf_2d[i, 1]), 
                ha='center', va='center', fontsize=10, fontweight='bold')
ax1.set_xlabel('Componente Principal 1', fontsize=12)
ax1.set_ylabel('Componente Principal 2', fontsize=12)
ax1.set_title('TF-IDF - Agrupamento por Categoria', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Plot 2: Sentence-BERT com conexões
draw_category_connections(ax2, sbert_2d, labels, cores_map)
ax2.scatter(sbert_2d[:, 0], sbert_2d[:, 1], c=cores, s=200, alpha=0.6, edgecolors='black', linewidth=1.5)
for i, txt in enumerate(range(len(textos))):
    ax2.annotate(f'{i}', (sbert_2d[i, 0], sbert_2d[i, 1]), 
                ha='center', va='center', fontsize=10, fontweight='bold')
ax2.set_xlabel('Componente Principal 1', fontsize=12)
ax2.set_ylabel('Componente Principal 2', fontsize=12)
ax2.set_title('Sentence-BERT - Agrupamento por Categoria', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

# Legenda
legend_elements = [Patch(facecolor=cor, label=cat, edgecolor='black') 
                   for cat, cor in cores_map.items()]
fig.legend(handles=legend_elements, loc='upper center', ncol=4, 
           fontsize=11, bbox_to_anchor=(0.5, 0.98), frameon=True)

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.savefig('agrupamento_categorias.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Visualização de agrupamento salva como 'agrupamento_categorias.png'")

## 11. Conclusões

### Principais Diferenças:

1. **Representação Vetorial:**
   - **TF-IDF**: Vetores esparsos baseados em frequência de palavras
   - **Sentence-BERT**: Vetores densos baseados em contexto semântico

2. **Agrupamento:**
   - **TF-IDF**: Agrupa textos com vocabulário similar
   - **Sentence-BERT**: Agrupa textos com significado similar

3. **Dimensionalidade:**
   - **TF-IDF**: Varia com o vocabulário (tipicamente centenas)
   - **Sentence-BERT**: Fixa em 384 dimensões (para o modelo usado)

4. **Aplicações:**
   - **TF-IDF**: Busca por palavras-chave, classificação simples
   - **Sentence-BERT**: Busca semântica, similaridade profunda, Q&A

### Vantagens e Desvantagens:

**TF-IDF:**
- ✅ Rápido e eficiente
- ✅ Interpretável (sabemos quais palavras são importantes)
- ❌ Não captura semântica profunda
- ❌ Problemas com sinônimos

**Sentence-BERT:**
- ✅ Captura significado profundo
- ✅ Funciona com sinônimos e paráfrases
- ❌ Mais lento (requer GPU para grandes volumes)
- ❌ Menos interpretável